In [1]:
!pip install datasets
!pip install textstat
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is i

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from datasets import load_dataset
import re
import numpy as np
from tqdm import tqdm
import torch
import nltk
import textstat
import random
import pickle
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import spacy
from collections import defaultdict

In [4]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 3.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
nlp = spacy.load("en_core_web_lg")

In [6]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [7]:
def get_dataset():
    train_dataset = load_dataset("openai/gsm8k", "main", split='train')
    test_dataset = load_dataset("openai/gsm8k", "main", split='test')
    return train_dataset, test_dataset

In [8]:
class Stats_Helpers():
  def __init__(self, question):
    self.question = question
    self.keywords = ['sum', 'total', 'difference', 'product', 'per']
    self.stopwords = set([
        'the', 'is', 'in', 'at', 'of', 'a', 'an', 'and', 'to', 'for', 'on',
        'with', 'as', 'by', 'that', 'from', 'it', 'this', 'be', 'or', 'are',
        'was', 'were', 'but', 'not', 'have', 'has', 'had', 'if', 'then', 'so'
    ])
    self.doc = nlp(question)
    self.unit_pattern = re.compile(r'\b(k|kg|km|m|cm|mm|ml|l|lb|oz|g|mg|hr|min|sec|miles?|feet|inches?)\b', re.IGNORECASE)

  def get_numeric_tokens(self):
    number_pattern = r'''
        \b              # Word boundary
        (?!\d{4}\b)     # Exclude 4-digit standalone numbers (likely years)
        \d{1,3}         # 1-3 digits
        (?:,\d{3})+     # Thousands separators (e.g., 1,000)
        (?:\.\d+)?      # Optional decimal portion
        |
        \d+             # Standard integers
        (?:\.\d+)?      # Decimals without commas
        \b              # Word boundary
    '''
    numbers = re.findall(number_pattern, self.question, re.VERBOSE)

    normalized = set()
    for num in numbers:
        try:
            normalized.add(float(num.replace(',', '')))
        except ValueError:
            continue

    return list(normalized)

  def get_keyword_frequencies(self):
    freq = {k: 0 for k in self.keywords}
    for keyword in self.keywords:
        freq[keyword] = len(re.findall(rf'\b{re.escape(keyword)}\b', self.question))
    return freq

  def get_stopword_ratio(self):
    words = re.findall(r'\b\w+\b', self.question)
    if not words: return 0.0
    stop_count = sum(1 for w in words if w in self.stopwords)
    return round(stop_count / len(words), 4)

  def get_pos_distribution(self):
    pos_counts = defaultdict(int)
    for token in self.doc:
        pos_counts[token.pos_] += 1

    total = len(self.doc)
    return {
        'prop_noun': pos_counts['NOUN'] / total if total else 0,
        'prop_verb': pos_counts['VERB'] / total if total else 0,
        'prop_num': pos_counts['NUM'] / total if total else 0,
        'prop_adj': pos_counts['ADJ'] / total if total else 0
    }

  def get_dependency_metrics(self):
    """Calculate syntactic complexity metrics from dependency parse"""
    def get_depth(token):
        if token.head == token:
            return 1
        return 1 + get_depth(token.head)

    depths = [get_depth(token) for token in self.doc]

    dep_distances = [abs(token.i - token.head.i) for token in self.doc
                    if token.head != token]

    return {
        'max_tree_depth': max(depths),
        'avg_dep_distance': sum(dep_distances)/len(dep_distances) if dep_distances else 0
    }

  def get_named_entities(self):
    """Extract quantities, units, and person names with spaCy NER"""
    entities = {
        'quantity_count': 0,
        'unit_count': 0,
        'person_count': 0,
        'numeric_entities': []
    }

    for ent in self.doc.ents:
        if ent.label_ in ['QUANTITY', 'CARDINAL', 'MONEY']:
            entities['quantity_count'] += 1
            try:
                entities['numeric_entities'].append(float(ent.text.replace(',', '')))
            except:
                continue
        elif ent.label_ == 'PERSON':
            entities['person_count'] += 1

    units = self.unit_pattern.findall(self.doc.text)
    entities['unit_count'] = len(units)

    return entities

def get_problem_stats(problem):
  stats = {}
  question = problem['question'].lower()
  stat_helper = Stats_Helpers(question)

  sentences = [s.strip() for s in question.split('.') if s.strip()]
  words = question.split()
  avg_word_length = sum(len(word) for word in words)/len(words) if words else 0
  stats['word_count'] = len(question.split())
  stats['sentence_count'] = len(sentences)
  stats['avg_sentence_length'] = np.mean([len(s.split()) for s in sentences])
  stats['avg_word_length'] = avg_word_length
  stats['flesch_reading_ease'] = textstat.flesch_reading_ease(question)   # 0 to 100, higher is more readable

  numeric_values = stat_helper.get_numeric_tokens()
  stats.update({
      'num_numeric_values': len(numeric_values),
      'min_value': min(numeric_values) if numeric_values else 0,
      'mean_value': np.mean(numeric_values) if numeric_values else 0,
      'max_value': max(numeric_values) if numeric_values else 0
  })
  stats['num_operations'] = re.findall(r'[+\-*/^]', question)

  stats['keyword_frequencies'] = stat_helper.get_keyword_frequencies()
  stats['stopword_ratio'] = stat_helper.get_stopword_ratio()

  stats.update(stat_helper.get_pos_distribution())
  stats.update(stat_helper.get_dependency_metrics())

  stats.update(stat_helper.get_named_entities())

  return stats

In [9]:
def get_model(model_name):
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
    )
    if model_name == "wizardmath":
        wizardmath_tokenizer = AutoTokenizer.from_pretrained("WizardLM/WizardMath-7B-V1.1")
        wizardmath_model = AutoModelForCausalLM.from_pretrained(
            "WizardLM/WizardMath-7B-V1.1",
            quantization_config=quantization_config,
            device_map={"": 0},
            torch_dtype=torch.float16
        )
        return {
            'model': wizardmath_model,
            'model_name': "wizardmath",
            'tokenizer': wizardmath_tokenizer,
            'cost_per_token': 0.7
        }
    elif model_name == "phi2":
        phi2_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
        phi2_model = AutoModelForCausalLM.from_pretrained(
            "microsoft/phi-2",
            quantization_config=quantization_config,
            device_map={"": 0},
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
        return {
            'model': phi2_model,
            'model_name': "phi2",
            'tokenizer': phi2_tokenizer,
            'cost_per_token': 0.27
        }

In [10]:
def extract_answer(answer_text):
    match = re.search(r'####\s*(-?\d+)', answer_text)
    if match:
        return match.group(1).strip()
    return None

In [11]:
def process_problem(problem, model_index, models):
    prompt = f"""

Follow these instructions:
1. Work through the problem step by step
2. Calculate the numerical answer
3. On the last line, write ONLY: #### <numerical answer>. Do not add any units like "kg" or "m", or any currency symbols like "$".
4. Do not write anything after the final answer

-------------------
EXAMPLE FORMAT:
Step 1: [explanation]
Step 2: [explanation]
Final calculation: [calculation]
#### [numerical answer]
-------------------

NOW SOLVE THE PROBLEM CORRECTLY: {problem['question']}
"""
    # print("Entered global process problem")
    model_obj = models[model_index]['model']
    tokenizer = models[model_index].get('tokenizer', None)
    if tokenizer:
        tokenizer = models[model_index]['tokenizer']

    # if models[model_index]['model_name'] == "wizardmath":
    inputs = tokenizer(prompt, return_tensors="pt").to(model_obj.device)
    outputs = model_obj.generate(
        inputs.input_ids,
        max_new_tokens=1024,
        temperature=0.1,
        do_sample=True,
        attention_mask=inputs.attention_mask,
        # pad_token_id=tokenizer.eos_token_id,
    )
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    prompt_end = full_output.find(f"NOW SOLVE THE PROBLEM CORRECTLY: {problem['question']}")
    if prompt_end != -1:
        prompt_end = prompt_end + len(f"NOW SOLVE THE PROBLEM CORRECTLY: {problem['question']}")
        model_response = full_output[prompt_end:].strip()
    else:
        model_response = full_output

    hash_match = re.search(r'####\s*([\$]?\s*\d+(?:\.\d+)?)', model_response)
    if hash_match:
        answer_text = hash_match.group(1)
        numeric_match = re.search(r'(\d+(?:\.\d+)?)', answer_text)
        if numeric_match:
            numeric_answer = numeric_match.group(1)
            # return f"{prompt}\n\n{model_response.split('####')[0].strip()}\n#### {numeric_answer}"
            return {
                'prompt': prompt,
                'response': model_response,
                'answer': numeric_answer
            }

    answer_match = re.search(r'(?:final answer|the answer is)[^0-9]*?([\$]?\s*\d+(?:\.\d+)?)',
                            model_response.lower())
    if answer_match:
        answer_text = answer_match.group(1)
        numeric_match = re.search(r'(\d+(?:\.\d+)?)', answer_text)
        if numeric_match:
            numeric_answer = numeric_match.group(1)
            answer_position = model_response.lower().find(answer_match.group(0))
            if answer_position != -1:
                # return f"{prompt}\n\n{model_response[:answer_position].strip()}\n#### {numeric_answer}"
                return {
                    'prompt': prompt,
                    'response': model_response,
                    'answer': numeric_answer
                }

    lines = model_response.split('\n')
    for i in range(len(lines)-1, max(0, len(lines)-5), -1):
        line = lines[i]
        if len(line.strip()) < 1 or any(word in line.lower() for word in ["step", "explanation"]):
            continue

        numeric_match = re.search(r'(\d+(?:\.\d+)?)', line)
        if numeric_match:
            numeric_answer = numeric_match.group(1)
            # return f"{prompt}\n\n{model_response.split(line)[0].strip()}\n#### {numeric_answer}"
            return {
                'prompt': prompt,
                'response': model_response,
                'answer': numeric_answer
            }

    # return full_output
    return {
        'prompt': prompt,
        'response': full_output,
        'answer': None
    }


In [12]:
import numpy as np
from scipy.linalg import solve_triangular
from collections import defaultdict

class LinUCB:
    def __init__(self, n_arms, context_size, alpha=0.5):
        self.n_arms = n_arms
        self.context_size = context_size
        self.alpha = alpha

        self.A = [np.eye(context_size) for _ in range(n_arms)]
        self.b = [np.zeros(context_size) for _ in range(n_arms)]
        self.theta = [np.zeros(context_size) for _ in range(n_arms)]

    def select_arm(self, context):
        scores = []
        for arm in range(self.n_arms):
            A_inv = np.linalg.inv(self.A[arm])
            self.theta[arm] = A_inv @ self.b[arm]

            score = self.theta[arm] @ context
            score += self.alpha * np.sqrt(context @ A_inv @ context)
            scores.append(score)

        return np.argmax(scores)

    def update(self, arm, context, reward):
        self.A[arm] += np.outer(context, context)
        self.b[arm] += reward * context

class ModelSwitcher:
    def __init__(self, models):
        self.models = models
        self.stats_helper = Stats_Helpers

        self.bandit = LinUCB(n_arms=2, context_size=23, alpha=0.25)

        self.feature_order = [
            'word_count', 'sentence_count', 'avg_sentence_length', 'avg_word_length',
            'flesch_reading_ease', 'num_numeric_values', 'min_value', 'mean_value',
            'max_value', 'stopword_ratio', 'prop_noun', 'prop_verb', 'prop_num',
            'prop_adj', 'max_tree_depth', 'avg_dep_distance', 'quantity_count',
            'unit_count', 'person_count', 'keyword_frequencies.sum',
            'keyword_frequencies.total', 'keyword_frequencies.difference',
            'keyword_frequencies.product'
        ]

    def _get_context_vector(self, problem_stats):
        context = []
        for feat in self.feature_order:
            if 'keyword' in feat:
                key = feat.split('.')[-1]
                context.append(problem_stats['keyword_frequencies'][key])
            else:
                context.append(problem_stats[feat])

        context = np.array(context)
        context = (context - np.mean(context)) / (np.std(context) + 1e-8)
        return context

    def process_problem(self, problem, correct_answer):
        # print("Entered model switcher process problem")
        stats = get_problem_stats(problem)
        # print("Got stats")
        context = self._get_context_vector(stats)
        # print("Got context")

        model_idx = self.bandit.select_arm(context)
        # print("Selected model")
        prediction = process_problem(problem, model_idx, self.models)
        # print(prediction)
        # print("Got prediction")

        is_correct = self._check_correctness(prediction, correct_answer)
        reward = self._calculate_reward(is_correct, model_idx)

        self.bandit.update(model_idx, context, reward)

        return prediction, model_idx, reward

    def _check_correctness(self, prediction, correct_answer):
        try:
            pred_answer = float(re.search(r'####\s*([\d.]+)', prediction).group(1))
            return np.isclose(pred_answer, correct_answer, rtol=0.01)
        except:
            return False

    def _calculate_reward(self, correct, model_idx):
        model_cost = self.models[model_idx]['cost_per_token']

        if correct:
            return 1
        else:
            return 0
        #     return 1.0 if model_idx == 0 else 0.5
        # else:
        #     return -2.0 if model_idx == 1 else -1.0

In [13]:
temp_set = get_dataset()
gsm8k_dataset = {
    'train': temp_set[0],
    'test': temp_set[1]
}
models = [get_model('phi2'), get_model('wizardmath')]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [14]:
def calculate_cost(prediction_data, model):
    """Calculate cost using structured prediction data"""
    tokenizer = model['tokenizer']
    cost_per_token = model['cost_per_token']

    input_tokens = tokenizer.encode(prediction_data['prompt'], return_tensors='pt').shape[1]
    output_tokens = tokenizer.encode(prediction_data['response'], return_tensors='pt').shape[1]

    return (input_tokens + output_tokens) * cost_per_token

In [ ]:
import time
import pandas as pd

switcher = ModelSwitcher(models)
switcher.bandit.alpha = 1

total_correct = 0
total_cost = 0

num_problems = 400
start_idx = 2000
cur_problem_idx = 0
# subset = gsm8k_dataset['train'].select(range(num_problems))
subset = gsm8k_dataset['train'].select(range(start_idx, start_idx + num_problems))

phi2_preds = []
wizardmath_preds = []

start_time = time.time()
for problem in tqdm(subset, desc="Processing problems"):
    cur_problem_idx += 1
    correct_answer = extract_answer(problem['answer'])

    prediction, model_idx, reward = switcher.process_problem(problem, correct_answer)

    # print(f"Problem: {problem['question']}")
    # print(f"Prediction: {prediction}")
    # print(f"Correct Answer: {correct_answer}")
    print(f"Model: {models[model_idx]['model_name']}")

    # Track performance
    predicted_answer = prediction['answer']
    # print(f"Predicted ans: {predicted_ans}")
    # print(f"Correct Answer: {correct_answer}")
    # if predicted_ans is not None and float(predicted_ans) == float(correct_answer):
    #     total_correct += 1
    # total_correct += (1 if switcher._check_correctness(prediction, correct_answer) else 0)
    total_cost += calculate_cost(prediction, models[model_idx])

    if predicted_answer is not None and float(predicted_answer) == float(correct_answer):
        total_correct += 1
        if model_idx == 0:
            phi2_preds.append({'question': problem['question'], 'answer': problem['answer'], 'model_response': prediction['response'], 'is_correct': True})
        else:
            wizardmath_preds.append({'question': problem['question'], 'answer': problem['answer'], 'model_response': prediction['response'], 'is_correct': True})
    else:
        if model_idx == 0:
            phi2_preds.append({'question': problem['question'], 'answer': problem['answer'], 'model_response': prediction['response'], 'is_correct': False})
        else:
            wizardmath_preds.append({'question': problem['question'], 'answer': problem['answer'], 'model_response': prediction['response'], 'is_correct': False})

    # if cur_problem_idx % 50 == 0:
    #     phi2_df = pd.DataFrame(phi2_preds)
    #     phi2_df['model_name'] = 'phi-2'

    #     wizardmath_df = pd.DataFrame(wizardmath_preds)
    #     wizardmath_df['model_name'] = 'wizardmath'

    #     combined_df = pd.concat([phi2_df, wizardmath_df], ignore_index=True)
    #     interim_csv_path = f'/content/gdrive/MyDrive/ai_project/combined_predictions_interim4_{cur_problem_idx}.csv'
    #     combined_df.to_csv(interim_csv_path, index=False)
    #     print(f"Interim predictions saved to: {interim_csv_path}")

    new_alpha = 1 - (cur_problem_idx/num_problems)
    if new_alpha < 0.1:
        new_alpha = 0.1
    switcher.bandit.alpha = new_alpha

end_time = time.time()

Processing problems:   0%|          | 1/400 [00:53<5:55:56, 53.53s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model: phi2


Processing problems:   0%|          | 2/400 [01:07<3:20:44, 30.26s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Model: wizardmath


Processing problems:   1%|          | 3/400 [01:59<4:26:15, 40.24s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Model: wizardmath


Processing problems:  41%|████▏     | 165/400 [1:26:21<1:13:50, 18.85s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Model: wizardmath


Processing problems:  42%|████▏     | 166/400 [1:27:14<1:53:35, 29.13s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Model: phi2


In [ ]:
import os
if not os.path.exists('/content/gdrive/MyDrive/ai_project'):
    os.makedirs('/content/gdrive/MyDrive/ai_project')

In [ ]:
import pandas as pd
import os

os.makedirs('/content', exist_ok=True)

phi2_df = pd.DataFrame(phi2_preds)
phi2_df['model_name'] = 'phi-2'

wizardmath_df = pd.DataFrame(wizardmath_preds)
wizardmath_df['model_name'] = 'wizardmath'

combined_df = pd.concat([phi2_df, wizardmath_df], ignore_index=True)
# combined_csv_path = '/content/combined_predictions.csv'
# combined_df.to_csv(combined_csv_path, index=False)
combined_csv_path = '/content/gdrive/MyDrive/ai_project/combined_predictions5.csv'
combined_df.to_csv(combined_csv_path, index=False)

accuracy = total_correct / num_problems
time_taken = end_time - start_time

# Create metrics DataFrame
metrics_df = pd.DataFrame({
    'total_problems': [num_problems],
    'correct_answers': [total_correct],
    'accuracy': [accuracy],
    'time_seconds': [time_taken],
    'avg_time_per_problem': [time_taken/num_problems],
    'total_cost': [total_cost]
})

# metrics_csv_path = '/content/experiment_metrics.csv'
metrics_csv_path = '/content/gdrive/MyDrive/ai_project/experiment_metrics4.csv'
metrics_df.to_csv(metrics_csv_path, index=False)

fpath = '/content/gdrive/MyDrive/ai_project/accuracy_summary4.txt'
# with open('/content/accuracy_summary.txt', 'w') as f:
with open(fpath, 'w') as f:
    f.write(f"""Experiment Results:
Total Problems: {num_problems}
Correct Answers: {total_correct}
Accuracy: {accuracy:.2%}
Time Taken: {time_taken:.2f} seconds
Average Time per Problem: {time_taken/num_problems:.2f}s
Total Cost: ${total_cost:.4f}
""")

print(f"Predictions saved to: {combined_csv_path}")
print(f"Metrics saved to: {metrics_csv_path}")
